# LangChain-based Course Planning System

This notebook implements an intelligent course planning system that:
- Analyzes whether a topic is broad or narrow
- Generates a single course for narrow topics
- Splits broad topics into multiple logically structured courses (max 8)
- Orders courses from beginner to advanced
- Ensures no overlap between courses
- Returns structured output using Pydantic models

## Install Required Packages

In [1]:
%pip install langchain langchain-ollama pydantic python-dotenv -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Dependencies

In [2]:
from typing import List
from pydantic import BaseModel, Field, field_validator
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

c:\Users\navee\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define Pydantic Models

These models ensure strict validation of the output structure.

In [3]:
class Course(BaseModel):
    """Represents a single course in the learning path."""
    course_name: str = Field(..., description="Name of the course")
    description: str = Field(..., description="Detailed description of the course")
    difficulty: str = Field(..., description="Difficulty level: Beginner, Intermediate, or Advanced")
    prerequisites: List[str] = Field(default_factory=list, description="List of prerequisite courses")
    
    @field_validator('difficulty')
    @classmethod
    def validate_difficulty(cls, v: str) -> str:
        """Ensure difficulty is one of the allowed values."""
        allowed = ['Beginner', 'Intermediate', 'Advanced']
        if v not in allowed:
            raise ValueError(f'Difficulty must be one of {allowed}')
        return v


class CoursePlan(BaseModel):
    """Represents a complete course plan with one or more courses."""
    is_broad: bool = Field(..., description="Whether the topic is broad and requires multiple courses")
    total_courses: int = Field(..., description="Total number of courses in the plan")
    courses: List[Course] = Field(..., description="List of courses in the plan")
    
    @field_validator('total_courses')
    @classmethod
    def validate_total_courses(cls, v: int, info) -> int:
        """Ensure total_courses matches the length of courses list."""
        if 'courses' in info.data:
            actual_count = len(info.data['courses'])
            if v != actual_count:
                raise ValueError(f'total_courses ({v}) must equal the number of courses ({actual_count})')
        return v
    
    @field_validator('courses')
    @classmethod
    def validate_max_courses(cls, v: List[Course]) -> List[Course]:
        """Ensure maximum 8 courses."""
        if len(v) > 8:
            raise ValueError('Maximum 8 courses allowed')
        if len(v) < 1:
            raise ValueError('At least 1 course is required')
        return v

## Create Course Planning System

This class uses LangChain with structured output to generate course plans.

In [4]:
class CoursePlanner:
    """LangChain-based course planning system."""
    
    def __init__(self, model_name: str = "phi3:mini", temperature: float = 0.7):
        """
        Initialize the course planner.
        
        Args:
            model_name: Name of the Ollama model to use
            temperature: Temperature for response generation (0.0-1.0)
        """
        self.llm = ChatOllama(model=model_name, temperature=temperature)
        self.structured_llm = self.llm.with_structured_output(CoursePlan)
        self.prompt = self._create_prompt()
        self.chain = self.prompt | self.structured_llm
    
    def _create_prompt(self) -> ChatPromptTemplate:
        """Create the prompt template for course planning."""
        system_message = """You are an expert curriculum designer and educational consultant.

Your task is to analyze a given topic and create a structured course plan.

ANALYSIS RULES:
1. Determine if the topic is BROAD or NARROW:
   - NARROW: A specific, focused topic that can be covered in a single comprehensive course
     Examples: "Introduction to Python Lists", "CSS Flexbox", "Linear Regression in Machine Learning"
   
   - BROAD: A wide-ranging topic that requires multiple courses to cover comprehensively
     Examples: "Machine Learning", "Web Development", "Data Science", "Artificial Intelligence"

2. For NARROW topics:
   - Create exactly 1 course
   - Set is_broad = false
   - Provide a comprehensive course covering the entire topic
   
3. For BROAD topics:
   - Create 2-8 courses (maximum 8)
   - Set is_broad = true
   - Split the topic into logical, non-overlapping courses
   - Order courses from Beginner → Intermediate → Advanced
   - Ensure each course builds upon previous ones
   - Assign appropriate prerequisites

COURSE STRUCTURE RULES:
- Each course must have: course_name, description, difficulty, prerequisites
- Difficulty MUST be EXACTLY one of these three values: "Beginner", "Intermediate", or "Advanced"
- DO NOT use combinations like "Beginner-Intermediate" or "Intermediate-Advanced"
- DO NOT create custom difficulty levels - use ONLY the three allowed values
- Prerequisites should reference course names from earlier courses in the plan
- First course should typically be "Beginner" with no prerequisites
- Courses should progress logically in difficulty
- Courses must NOT overlap in content
- Each course should cover a distinct subset of the broader topic

OUTPUT RULES:
- total_courses MUST equal the actual number of courses
- Maximum 8 courses allowed
- Do NOT add any extra fields
- Ensure all required fields are present
- difficulty field accepts ONLY: "Beginner", "Intermediate", or "Advanced" - no other values allowed"""

        user_message = """Create a course plan for the following topic:

Course Title: {course_title}
Course Description: {course_description}

Analyze whether this topic is broad or narrow, then create an appropriate course plan."""

        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            ("user", user_message)
        ])
    
    def create_course_plan(self, course_title: str, course_description: str) -> CoursePlan:
        """
        Generate a course plan based on the input.
        
        Args:
            course_title: The title of the topic
            course_description: A description of what should be covered
            
        Returns:
            CoursePlan object with structured course information
        """
        result = self.chain.invoke({
            "course_title": course_title,
            "course_description": course_description
        })
        
        return result
    
    def print_course_plan(self, plan: CoursePlan) -> None:
        """
        Pretty print the course plan.
        
        Args:
            plan: CoursePlan object to display
        """
        print("=" * 80)
        print(f"COURSE PLAN ANALYSIS")
        print("=" * 80)
        print(f"Topic is: {'BROAD' if plan.is_broad else 'NARROW'}")
        print(f"Total Courses: {plan.total_courses}")
        print("=" * 80)
        print()
        
        for idx, course in enumerate(plan.courses, 1):
            print(f"📚 COURSE {idx}: {course.course_name}")
            print(f"   Difficulty: {course.difficulty}")
            print(f"   Description: {course.description}")
            if course.prerequisites:
                print(f"   Prerequisites: {', '.join(course.prerequisites)}")
            else:
                print(f"   Prerequisites: None")
            print()
        print("=" * 80)

## Initialize the Course Planner

In [5]:
planner = CoursePlanner(model_name="llama3:8b", temperature=0.7)

## Example 1: Narrow Topic (Single Course)

In [6]:
# Example of a narrow topic that should result in a single course
narrow_plan = planner.create_course_plan(
    course_title="Introduction to Python Decorators",
    course_description="Learn how to use and create decorators in Python, including function decorators, class decorators, and decorator patterns."
)

planner.print_course_plan(narrow_plan)

COURSE PLAN ANALYSIS
Topic is: NARROW
Total Courses: 1

📚 COURSE 1: Introduction to Python Decorators
   Difficulty: Intermediate
   Description: Learn how to use and create decorators in Python, including function decorators, class decorators, and decorator patterns.
   Prerequisites: None



## Example 2: Broad Topic (Multiple Courses)

In [7]:
# Example of a broad topic that should result in multiple courses
broad_plan = planner.create_course_plan(
    course_title="Machine Learning",
    course_description="Comprehensive coverage of machine learning concepts, algorithms, and applications including supervised learning, unsupervised learning, deep learning, and practical implementations."
)

planner.print_course_plan(broad_plan)

COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 6

📚 COURSE 1: Introduction to Machine Learning
   Difficulty: Beginner
   Description: Foundational concepts in machine learning: supervised and unsupervised learning, regression, classification, clustering.
   Prerequisites: None

📚 COURSE 2: Supervised Learning Fundamentals
   Difficulty: Intermediate
   Description: Regression, classification, decision trees, random forests, support vector machines (SVMs), logistic regression.
   Prerequisites: Introduction to Machine Learning

📚 COURSE 3: Unsupervised Learning and Dimensionality Reduction
   Difficulty: Intermediate
   Description: K-means clustering, hierarchical clustering, principal component analysis (PCA), t-distributed Stochastic Neighbor Embedding (t-SNE), autoencoders.
   Prerequisites: Introduction to Machine Learning, Supervised Learning Fundamentals

📚 COURSE 4: Deep Learning and Neural Networks
   Difficulty: Advanced
   Description: Feedforward networks, convolutiona

## Example 3: Another Broad Topic

In [8]:
# Another broad topic example
web_dev_plan = planner.create_course_plan(
    course_title="Full Stack Web Development",
    course_description="Complete guide to becoming a full stack web developer, covering frontend technologies, backend development, databases, deployment, and modern web development practices."
)

planner.print_course_plan(web_dev_plan)

COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 6

📚 COURSE 1: Introduction to Frontend Development
   Difficulty: Beginner
   Description: Learn the basics of frontend development, including HTML5, CSS3, JavaScript, and modern frameworks like React or Angular.
   Prerequisites: None

📚 COURSE 2: Frontend Frameworks and Libraries
   Difficulty: Intermediate
   Description: Dive deeper into popular frontend frameworks and libraries such as React, Angular, Vue.js, and learn how to use them effectively in real-world projects.
   Prerequisites: Introduction to Frontend Development

📚 COURSE 3: Backend Fundamentals
   Difficulty: Intermediate
   Description: Learn the basics of backend development using popular frameworks such as Node.js, Ruby on Rails, Django, and learn about database integration and API design.
   Prerequisites: Introduction to Frontend Development, Frontend Frameworks and Libraries

📚 COURSE 4: Backend Frameworks and Databases
   Difficulty: Advanced
   Description: 

## Validate the Output Structure

In [9]:
# Validate the structure of the generated plan
print("Validation Results:")
print("=" * 80)

# Check narrow topic
print(f"\n✓ Narrow topic plan:")
print(f"  - is_broad: {narrow_plan.is_broad} (Expected: False)")
print(f"  - total_courses: {narrow_plan.total_courses} (Expected: 1)")
print(f"  - Actual courses: {len(narrow_plan.courses)}")
print(f"  - Match: {narrow_plan.total_courses == len(narrow_plan.courses)}")

# Check broad topic
print(f"\n✓ Broad topic plan:")
print(f"  - is_broad: {broad_plan.is_broad} (Expected: True)")
print(f"  - total_courses: {broad_plan.total_courses}")
print(f"  - Actual courses: {len(broad_plan.courses)}")
print(f"  - Match: {broad_plan.total_courses == len(broad_plan.courses)}")
print(f"  - Within limit (1-8): {1 <= len(broad_plan.courses) <= 8}")

# Validate difficulties
print(f"\n✓ Difficulty validation:")
valid_difficulties = ['Beginner', 'Intermediate', 'Advanced']
for course in broad_plan.courses:
    is_valid = course.difficulty in valid_difficulties
    print(f"  - {course.course_name}: {course.difficulty} ({'✓' if is_valid else '✗'})")

print("\n" + "=" * 80)

Validation Results:

✓ Narrow topic plan:
  - is_broad: False (Expected: False)
  - total_courses: 1 (Expected: 1)
  - Actual courses: 1
  - Match: True

✓ Broad topic plan:
  - is_broad: True (Expected: True)
  - total_courses: 6
  - Actual courses: 6
  - Match: True
  - Within limit (1-8): True

✓ Difficulty validation:
  - Introduction to Machine Learning: Beginner (✓)
  - Supervised Learning Fundamentals: Intermediate (✓)
  - Unsupervised Learning and Dimensionality Reduction: Intermediate (✓)
  - Deep Learning and Neural Networks: Advanced (✓)
  - Specialized Topics in Machine Learning: Advanced (✓)
  - Machine Learning Applications and Project Development: Advanced (✓)



## Export to JSON

You can export the course plan to JSON format for API integration.

In [10]:
# Convert to JSON
json_output = broad_plan.model_dump_json(indent=2)
print("JSON Output:")
print(json_output)

# You can also convert to dict
dict_output = broad_plan.model_dump()
print("\n\nDictionary Output:")
print(dict_output)

JSON Output:
{
  "is_broad": true,
  "total_courses": 6,
  "courses": [
    {
      "course_name": "Introduction to Machine Learning",
      "description": "Foundational concepts in machine learning: supervised and unsupervised learning, regression, classification, clustering.",
      "difficulty": "Beginner",
      "prerequisites": []
    },
    {
      "course_name": "Supervised Learning Fundamentals",
      "description": "Regression, classification, decision trees, random forests, support vector machines (SVMs), logistic regression.",
      "difficulty": "Intermediate",
      "prerequisites": [
        "Introduction to Machine Learning"
      ]
    },
    {
      "course_name": "Unsupervised Learning and Dimensionality Reduction",
      "description": "K-means clustering, hierarchical clustering, principal component analysis (PCA), t-distributed Stochastic Neighbor Embedding (t-SNE), autoencoders.",
      "difficulty": "Intermediate",
      "prerequisites": [
        "Introduction 

## Interactive Usage

Use this cell to test with your own topics!

In [11]:
# Try your own topic here!
custom_plan = planner.create_course_plan(
    course_title="Your Topic Here",  # Change this
    course_description="Your description here"  # Change this
)

planner.print_course_plan(custom_plan)

COURSE PLAN ANALYSIS
Topic is: NARROW
Total Courses: 1

📚 COURSE 1: Introduction to Computational Thinking
   Difficulty: Beginner
   Description: This comprehensive course introduces students to the fundamental concepts of computational thinking and problem-solving. Students will learn about algorithms, data structures, and programming languages, with a focus on developing logical thinking and analytical skills.
   Prerequisites: None



## Save Course Plan for Use in Syllabus Generation

Export the first course from a broad plan to use in the syllabus generation notebook.

In [12]:
# Generate a course plan and save the first course
import pickle
import os

# Create a course plan
my_course_plan = planner.create_course_plan(
    course_title="Data Science",
    course_description="Learn data science from fundamentals to advanced applications including statistics, programming, machine learning, and data visualization."
)

planner.print_course_plan(my_course_plan)

# Save the first course for syllabus generation
first_course = my_course_plan.courses[0]

# Create output directory
os.makedirs("output", exist_ok=True)

# Save as pickle for easy loading
with open("output/selected_course.pkl", "wb") as f:
    pickle.dump(first_course, f)

# Also save as JSON
with open("output/selected_course.json", "w") as f:
    f.write(first_course.model_dump_json(indent=2))

print(f"\n✓ First course saved:")
print(f"  - Course: {first_course.course_name}")
print(f"  - Difficulty: {first_course.difficulty}")
print(f"  - Saved to: output/selected_course.pkl and output/selected_course.json")

COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 6

📚 COURSE 1: Introduction to Data Science
   Difficulty: Beginner
   Description: Learn the fundamentals of data science including data types, cleaning, and visualization techniques.
   Prerequisites: None

📚 COURSE 2: Statistics for Data Science
   Difficulty: Intermediate
   Description: Master statistical concepts essential for data analysis and modeling, including hypothesis testing, confidence intervals, and regression analysis.
   Prerequisites: Introduction to Data Science

📚 COURSE 3: Programming for Data Science
   Difficulty: Intermediate
   Description: Learn programming skills in languages such as Python, R, or SQL, necessary for data manipulation, analysis, and visualization.
   Prerequisites: Statistics for Data Science, Introduction to Data Science

📚 COURSE 4: Machine Learning Fundamentals
   Difficulty: Intermediate
   Description: Explore the basics of machine learning including supervised and unsupervised learning